In [52]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

### Raw Data
The data was scraped from the season summary pages from [Basketball Reference](basketball-reference.com). It catalogues every team's advanced stats from the 1977 (The first merged ABA-NBA season) season to the 2021 season. It has missing values as well as some completely empty columns.

In [53]:
raw_data = pd.read_csv('../data/teams.csv', thousands=',')
raw_data.head()

,Unnamed: 0,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,.1,eFG%.1,TOV%.1,DRB%,FT/FGA.1,.2,Arena,Attend.,Attend./G,Year
0,0,Portland Trail Blazers*,24.5,49,33,55,27,5.59,-0.20,5.39,...,NaN,0.460,17.2,69.3,0.255,NaN,Memorial Coliseum,NaN,NaN,1977
1,1,Denver Nuggets*,27.3,50,32,54,28,5.22,-0.27,4.95,...,NaN,0.463,18.2,68.0,0.211,NaN,McNichols Sports Arena,NaN,NaN,1977
2,2,Philadelphia 76ers*,25.6,50,32,51,31,3.94,-0.16,3.78,...,NaN,0.451,16.7,66.0,0.197,NaN,The Spectrum,NaN,NaN,1977
3,3,Golden State Warriors*,25.8,46,36,49,33,3.22,-0.12,3.10,...,NaN,0.470,17.2,67.8,0.224,NaN,Oakland-Alameda County Coliseum Arena,NaN,NaN,1977
4,4,Los Angeles Lakers*,27.2,53,29,48,34,2.72,-0.08,2.64,...,NaN,0.452,15.6,66.1,0.194,NaN,The Forum,501434.0,NaN,1977


### Data Cleaning
Including only the columns with actual data gets rid of the completely null columns.

In [54]:
filtered = raw_data[['Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg', 'DRtg', 'NRtg', 'Pace',
'FTr', '3PAr', 'TS%', "eFG%", 'TOV%', 'ORB%', "FT/FGA", 'eFG%.1', 'TOV%.1', 'DRB%', 'FT/FGA.1',"Arena", "Attend.", "Attend./G", 'Year']]

filtered.head()

,Team,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,...,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Arena,Attend.,Attend./G,Year
0,Portland Trail Blazers*,24.5,49,33,55,27,5.59,-0.20,5.39,103.2,...,33.4,0.254,0.460,17.2,69.3,0.255,Memorial Coliseum,NaN,NaN,1977
1,Denver Nuggets*,27.3,50,32,54,28,5.22,-0.27,4.95,100.7,...,34.2,0.275,0.463,18.2,68.0,0.211,McNichols Sports Arena,NaN,NaN,1977
2,Philadelphia 76ers*,25.6,50,32,51,31,3.94,-0.16,3.78,101.2,...,34.6,0.275,0.451,16.7,66.0,0.197,The Spectrum,NaN,NaN,1977
3,Golden State Warriors*,25.8,46,36,49,33,3.22,-0.12,3.10,102.4,...,33.0,0.211,0.470,17.2,67.8,0.224,Oakland-Alameda County Coliseum Arena,NaN,NaN,1977
4,Los Angeles Lakers*,27.2,53,29,48,34,2.72,-0.08,2.64,101.4,...,31.0,0.188,0.452,15.6,66.1,0.194,The Forum,501434.0,NaN,1977


### Renaming Columns
The data set has identical column names for both defensive and offensive metric. So now, we give them distinct names.

In [55]:
renamed = filtered.rename(mapper={"eFG%.1": "OeFG%", "TOV%.1": "OTOV%", "FT/FGA.1": "OFT/FGA"}, axis='columns')
renamed.head()

,Team,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,...,ORB%,FT/FGA,OeFG%,OTOV%,DRB%,OFT/FGA,Arena,Attend.,Attend./G,Year
0,Portland Trail Blazers*,24.5,49,33,55,27,5.59,-0.20,5.39,103.2,...,33.4,0.254,0.460,17.2,69.3,0.255,Memorial Coliseum,NaN,NaN,1977
1,Denver Nuggets*,27.3,50,32,54,28,5.22,-0.27,4.95,100.7,...,34.2,0.275,0.463,18.2,68.0,0.211,McNichols Sports Arena,NaN,NaN,1977
2,Philadelphia 76ers*,25.6,50,32,51,31,3.94,-0.16,3.78,101.2,...,34.6,0.275,0.451,16.7,66.0,0.197,The Spectrum,NaN,NaN,1977
3,Golden State Warriors*,25.8,46,36,49,33,3.22,-0.12,3.10,102.4,...,33.0,0.211,0.470,17.2,67.8,0.224,Oakland-Alameda County Coliseum Arena,NaN,NaN,1977
4,Los Angeles Lakers*,27.2,53,29,48,34,2.72,-0.08,2.64,101.4,...,31.0,0.188,0.452,15.6,66.1,0.194,The Forum,501434.0,NaN,1977


#### Missing Values
The missing values are all located in the 3PAr column, which measures the proportion between 3-point and 2-point shot attempts (The 3-point shot was only introduced in 1979). So any missing values will default to 0. The other missing values are in the attendence columns. 

In [56]:
renamed.isnull().sum()

Team           0
Age            0
W              0
L              0
PW             0
PL             0
MOV            0
SOS            0
SRS            0
ORtg           0
DRtg           0
NRtg           0
Pace           0
FTr            0
3PAr          66
TS%            0
eFG%           0
TOV%           0
ORB%           0
FT/FGA         0
OeFG%          0
OTOV%          0
DRB%           0
OFT/FGA        0
Arena          0
Attend.       80
Attend./G    496
Year           0
dtype: int64

In [57]:
renamed['3PAr'].fillna(0, inplace = True)
renamed.head()

,Team,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,...,ORB%,FT/FGA,OeFG%,OTOV%,DRB%,OFT/FGA,Arena,Attend.,Attend./G,Year
0,Portland Trail Blazers*,24.5,49,33,55,27,5.59,-0.20,5.39,103.2,...,33.4,0.254,0.460,17.2,69.3,0.255,Memorial Coliseum,NaN,NaN,1977
1,Denver Nuggets*,27.3,50,32,54,28,5.22,-0.27,4.95,100.7,...,34.2,0.275,0.463,18.2,68.0,0.211,McNichols Sports Arena,NaN,NaN,1977
2,Philadelphia 76ers*,25.6,50,32,51,31,3.94,-0.16,3.78,101.2,...,34.6,0.275,0.451,16.7,66.0,0.197,The Spectrum,NaN,NaN,1977
3,Golden State Warriors*,25.8,46,36,49,33,3.22,-0.12,3.10,102.4,...,33.0,0.211,0.470,17.2,67.8,0.224,Oakland-Alameda County Coliseum Arena,NaN,NaN,1977
4,Los Angeles Lakers*,27.2,53,29,48,34,2.72,-0.08,2.64,101.4,...,31.0,0.188,0.452,15.6,66.1,0.194,The Forum,501434.0,NaN,1977


To clean the other columns, we can estimate that total attendence is approximately 40 times the attendence per game, which replicates uneven attendance on weekends and weekdays. Otherwise, we have to drop the rows that don't have either value.

In [58]:
dropped = renamed.dropna(subset=['Attend.', 'Attend./G'], how='all')

dropped['Attend.'] = dropped.apply(
  lambda row: 40 * row['Attend./G'] if pd.isnull(row['Attend.']) else row['Attend.'],
  axis = 1
)

dropped['Attend./G'] = dropped.apply(
  lambda row: row['Attend.'] / 40 if pd.isnull(row['Attend./G']) else row['Attend./G'],
  axis = 1
)

dropped.head()

,Team,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,...,ORB%,FT/FGA,OeFG%,OTOV%,DRB%,OFT/FGA,Arena,Attend.,Attend./G,Year
4,Los Angeles Lakers*,27.2,53,29,48,34,2.72,-0.08,2.64,101.4,...,31.0,0.188,0.452,15.6,66.1,0.194,The Forum,501434.0,12535.850,1977
16,Boston Celtics*,28.7,44,38,36,46,-1.98,0.08,-1.90,96.5,...,31.1,0.212,0.450,13.4,72.8,0.204,Boston Garden,517391.0,12934.775,1977
17,New Orleans Jazz,26.1,35,47,33,49,-2.82,0.13,-2.68,97.0,...,31.0,0.222,0.452,15.5,68.2,0.238,Louisiana Superdome,441320.0,11033.000,1977
26,Los Angeles Lakers*,25.8,45,37,48,34,2.67,-0.08,2.59,103.3,...,30.4,0.205,0.463,15.2,66.0,0.194,The Forum,534017.0,13350.425,1978
39,Boston Celtics,29.7,32,50,36,46,-1.95,0.09,-1.86,99.1,...,32.4,0.220,0.456,13.9,71.4,0.226,Boston Garden,437937.0,10948.425,1978


#### Finalizing the Dataset
The source site of the data put a * on an team that made the postseason, since this serves no real purpose here, it is easy to remove them. 

In [59]:
final = dropped.replace('\*','',regex=True).reset_index(drop=True)
final.to_csv('../data/teams_processed.csv', index=False)
final.head()

,Team,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,...,ORB%,FT/FGA,OeFG%,OTOV%,DRB%,OFT/FGA,Arena,Attend.,Attend./G,Year
0,Los Angeles Lakers,27.2,53,29,48,34,2.72,-0.08,2.64,101.4,...,31.0,0.188,0.452,15.6,66.1,0.194,The Forum,501434.0,12535.850,1977
1,Boston Celtics,28.7,44,38,36,46,-1.98,0.08,-1.90,96.5,...,31.1,0.212,0.450,13.4,72.8,0.204,Boston Garden,517391.0,12934.775,1977
2,New Orleans Jazz,26.1,35,47,33,49,-2.82,0.13,-2.68,97.0,...,31.0,0.222,0.452,15.5,68.2,0.238,Louisiana Superdome,441320.0,11033.000,1977
3,Los Angeles Lakers,25.8,45,37,48,34,2.67,-0.08,2.59,103.3,...,30.4,0.205,0.463,15.2,66.0,0.194,The Forum,534017.0,13350.425,1978
4,Boston Celtics,29.7,32,50,36,46,-1.95,0.09,-1.86,99.1,...,32.4,0.220,0.456,13.9,71.4,0.226,Boston Garden,437937.0,10948.425,1978


### Evaluation of Data Cleaning
By comparing the size of the first and final dataframes, we can get a good idea of how much data was wasted versus how much data was viable.

In [60]:
print("Shape of raw data: ", raw_data.shape)
print("Shape of cleaned data: ", final.shape)
rows_lost = len(raw_data.index) - len(final.index)
percentage_lost = round((rows_lost / len(raw_data.index)) * 100, 2)
print(rows_lost, "rows lost")
print(str(percentage_lost) + "% of data lost")

Shape of raw data:  (1200, 32)
Shape of cleaned data:  (1133, 28)
67 rows lost
5.58% of data lost
